In [ ]:
# Install necessary libraries
!pip install openai pinecone-client sentence-transformers pandas

In [ ]:
# Import libraries
import openai
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import pandas as pd
from google.colab import files

# Initialize OpenAI API key
openai.api_key = "pcsk_2z3Eyk_SbfQ6dQdBwCq1oJgoenG3i8CmzqirYEhjwQNKQy1U8AwpissfLtThKhackyAd7P"

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_4EU7iH_GS7g1yetLN6YG2NcQeMBeGMLfnQRPGmUUAqVUdgM79Km9ZveCmeDf3YcFvsBpbG")

# Specify the environment (e.g., AWS)
spec = ServerlessSpec(
    cloud="aws",
    region="us-west-2"  # Adjust region as needed
)

# Check if the index exists; create it if it doesn't
index_name = "business-qa"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Embedding dimension size
        metric="cosine",
        spec=spec
    )

# Connect to the index
index = pc.index(index_name)

# Upload the dataset
uploaded = files.upload()  # Upload a file named 'data.csv'

# Load and preprocess data
def preprocess_data(file_path):
    data = pd.read_csv(file_path)
    chunks = []
    for idx, text in enumerate(data['content']):
        words = text.split()
        for i in range(0, len(words), 300):  # Split text into chunks of 300 words
            chunk = " ".join(words[i:i + 300])
            chunks.append((f"{idx}-{i//300}", chunk))
    return chunks

file_path = next(iter(uploaded))  # Get uploaded file name
chunks = preprocess_data(file_path)
print(f"Total Chunks Processed: {len(chunks)}")

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for chunks
embeddings = model.encode([chunk[1] for chunk in chunks], show_progress_bar=True)

# Prepare and upload data to Pinecone
to_upsert = [(chunk[0], emb, {"text": chunk[1]}) for chunk, emb in zip(chunks, embeddings)]
index.upsert(to_upsert)
print(f"Uploaded {len(to_upsert)} embeddings to Pinecone.")

# Define the Retrieval Augmented Generation (RAG) pipeline
def rag_pipeline(query, top_k=5):
    # Generate embedding for the query
    query_emb = model.encode([query])[0]

    # Retrieve relevant documents from Pinecone
    results = index.query(query_emb, top_k=top_k, include_metadata=True)
    context = " ".join([match['metadata']['text'] for match in results['matches']])

    # Generate response using OpenAI
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = openai.Completion.create(
        model="gpt-4",
        prompt=prompt,
        temperature=0.7,
        max_tokens=300
    )
    return response['choices'][0]['text'].strip()

# Test the RAG QA Bot
query = "How do I update my account details?"
answer = rag_pipeline(query)
print(f"Q: {query}\nA: {answer}")
